# **Image Reconstruction**

## Setup Environment

In [1]:
%load_ext autoreload
%autoreload 2
import os

# Test if it's running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:  # If it's running in Google Colab, mount the drive
    from google.colab import drive
    drive.mount('/content/drive')
    HOME = "/content/drive/MyDrive/Image Restoration/image-restoration"
    if not os.path.exists(HOME): # If the repo isn't cloned yet, clone it
        os.chdir("/content/drive/MyDrive/Image Restoration/")
        !git clone https://github.com/davegabe/image-restoration.git
        os.chdir(HOME)
    else: # If it's already cloned, update it
        os.chdir(HOME)
        !git pull
else:  # If it's running in local machine, use the local path
    HOME = "./"
    %cd "{HOME}"

TRAINING_PATH = HOME + "Dataset/Training/"
EVALUATION_PATH = HOME + "Dataset/Evaluation/"
RESULTS_PATH = HOME + "Results/"
MODEL_PATH = HOME + "Model/"

## Data Download

In [ ]:
from data.downloader import download
from data.corrupter import corrupt
print("Generating dataset...")
download(TRAINING_PATH + "Original", EVALUATION_PATH + "Original")
corrupt(TRAINING_PATH + "Original", EVALUATION_PATH + "Original", TRAINING_PATH + "Corrupted", EVALUATION_PATH + "Corrupted")
print("Dataset generated.")

## Training

In [ ]:
from autoencoder.train import train
train(TRAINING_PATH + "Original", TRAINING_PATH + "Corrupted", MODEL_PATH)

## Testing

In [ ]:
from autoencoder.test import test
TRAINING_PATH = HOME + "Dataset/Training/"
EVALUATION_PATH = HOME + "Dataset/Evaluation/"
test(MODEL_PATH, EVALUATION_PATH + "Original", RESULTS_PATH)

In [ ]:
from IPython.display import Image
# Plot the corrupted image
#Image(filename=EVALUATION_PATH + "Corrupted/0.png")
# Plot the original image
#Image(filename=EVALUATION_PATH + "Original/0.png")
# Plot the reconstructed image
#Image(filename=RESULTS_PATH + "0.png")